In [1]:
# NEED TO RUN ON CLUSTER
import sys
CLUSTER=True
if CLUSTER:
    sys.path.insert(0, '/if6/nb2cz/anaconda/lib/python2.7/site-packages')

import  os

import numpy as np
import pandas as pd


from create_df_larger import read_df_larger
df, dfc, all_homes, appliance_min, national_average = read_df_larger()

df = df.rename(columns={'house_num_rooms':'num_rooms',
                        'num_occupants':'total_occupants',
                        'difference_ratio_min_max':'ratio_difference_min_max'})
K_min, K_max = 1,6
F_min, F_max=1,8

from all_functions import *
from features_larger import *

import sys

from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import LeaveOneOut

NUM_NEIGHBOUR_MAX = 6
F_MAX = 6

import json



from sklearn.cross_validation import LeaveOneOut
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from collections import OrderedDict

In [23]:
out={}
for appliance in ["hvac","fridge","wm","dw"]:
    out[appliance]={}
    if appliance=="hvac":
        start, stop=5, 11
    else:
        start, stop=1,13
    appliance_df = df.ix[all_homes[appliance]]
    for home in appliance_df.index:
        train_d = appliance_df[~appliance_df.index.isin([home])]
        pred_df = train_d[['%s_%d' %(appliance, month) for month in range(start, stop)]].mean()
        gt = appliance_df.ix[home][['%s_%d' %(appliance, month) for month in range(start, stop)]]
        error = (gt-pred_df).abs()
        #print error
        percentage_error = error*100/gt
        #print percentage_error
        percentage_error[percentage_error>100]=100


        percentage_accuracy = 100-percentage_error
        out[appliance][home] = percentage_error.mean()

In [24]:
pd.Series(out['hvac']).mean()

45.8037084044808

In [25]:
pd.DataFrame(out).mean()

dw        60.903677
fridge    24.570173
hvac      45.803708
wm        54.703832
dtype: float64